In [2]:
import os 
import pandas as pd
import numpy as np
import pickle


In [3]:

# load datasets
LERTA1 = pd.read_csv('../../data/LERTA/CLEAN_Lerta_House1.csv', sep=',')
LETRA2 = pd.read_csv('../../data/LERTA/CLEAN_Lerta_House2.csv', sep=',')
LETRA3 = pd.read_csv('../../data/LERTA/CLEAN_Lerta_House3.csv', sep=',')
LETRA4 = pd.read_csv('../../data/LERTA/CLEAN_Lerta_House4.csv', sep=',')


In [4]:
LERTA1.head()

# write me a for loop to sum from 1 to 10


,Time,AGGREGATE,5c600454ff5adf000142cdc8,5c60050bff5adf000142cdc9,KETTLE,5c60062eff5adf000142cdcc,COOKER,FRIDGE,MICROWAVE,5c61dcdeff5adf000142cdd4,IRON
0,2020-02-27 00:00:36+00:00,64.0,0.0,0.0,0.0,0.0,0.0,42.0,0.0,0.0,0.0
1,2020-02-27 00:00:42+00:00,64.0,0.0,0.0,0.0,0.0,0.0,42.0,0.0,0.0,0.0
2,2020-02-27 00:00:48+00:00,64.0,0.0,0.0,0.0,0.0,0.0,42.0,0.0,0.0,0.0
3,2020-02-27 00:00:54+00:00,64.0,0.0,0.0,0.0,0.0,0.0,42.0,0.0,0.0,0.0
4,2020-02-27 00:01:00+00:00,64.0,0.0,0.0,0.0,0.0,0.0,42.0,0.0,0.0,0.0


In [20]:
# time to datetime
LERTA1['Time'] = pd.to_datetime(LERTA1['Time'], format='%Y-%m-%d %H:%M:%S')
LETRA2['Time'] = pd.to_datetime(LETRA2['Time'], format='%Y-%m-%d %H:%M:%S')
LETRA3['Time'] = pd.to_datetime(LETRA3['Time'], format='%Y-%m-%d %H:%M:%S')
LETRA4['Time'] = pd.to_datetime(LETRA4['Time'], format='%Y-%m-%d %H:%M:%S')

# set index
LERTA1 = LERTA1.set_index('Time')
LETRA2 = LETRA2.set_index('Time')
LETRA3 = LETRA3.set_index('Time')
LETRA4 = LETRA4.set_index('Time')



In [21]:
# conversion to kWh
LETRA1 = watts2kwh(LERTA1)
LETRA2 = watts2kwh(LETRA2)
LETRA3 = watts2kwh(LETRA3)
LETRA4 = watts2kwh(LETRA4)

In [25]:
LETRA1["AGGREGATE"].max()

0.013128333333333334

In [38]:
# resample to given interval
def resample2interval(interval, df : pd.DataFrame):
    df = df.resample(interval).sum()
    return df
def watts2kwh(df):
    # converts from W to kWh 6 second intervals
    df = (df/1000)  * (6/3600)
    return df

def calculate_loadprofiles(df):
    # resample to daily and hourly
    hourly = df.resample('H').sum()
    daily = df.resample('D').sum()


        
    # daily load profile
    loadprofile_daily = hourly.groupby(hourly.index.hour).mean()

    # weekly load profile
    loadprofile_weekly = daily.groupby(daily.index.dayofweek).mean()

    # monthly load profile
    loadprofile_monthly = daily.groupby(daily.index.day).mean()

    # save to dictioanry
    loadprofiles = {
        "daily": loadprofile_daily,
        "weekly": loadprofile_weekly,
        "monthly": loadprofile_monthly
    }
    return loadprofiles



In [41]:
# calculate load profiles and save them to dictionary
houses_loadprofiles = {
    "house1": calculate_loadprofiles(LETRA1),
    "house2": calculate_loadprofiles(LETRA2),
    "house3": calculate_loadprofiles(LETRA3),
    "house4": calculate_loadprofiles(LETRA4)
}

#  save to pickle
with open('../../data/LERTA/loadprofiles.pickle', 'wb') as handle:
    pickle.dump(houses_loadprofiles, handle, protocol=pickle.HIGHEST_PROTOCOL)




